In [1]:
import pandas as pd 
import numpy as np

In [3]:
df=pd.read_csv("Natixis.csv",sep=";")

In [5]:
df.head()

,Date,Cours de Natixis
0,02/01/2015,"5,621"
1,05/01/2015,"5,424"
2,06/01/2015,"5,329"
3,07/01/2015,"5,224"
4,08/01/2015,"5,453"


## Compute the returns : 

(Pt+1 - Pt ) / Pt

In [8]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
df['Cours de Natixis'] = (
    df['Cours de Natixis']
    .astype(str)                       # transformer en string pour manipuler
    .str.replace("€", "", regex=False) # enlever symbole euro si présent
    .str.replace(" ", "", regex=False) # enlever les espaces
    .str.replace(",", ".", regex=False) # remplacer virgule par point
)

df['Cours de Natixis'] = pd.to_numeric(df['Cours de Natixis'], errors='coerce')
df.head()

,Date,Cours de Natixis
0,2015-01-02,5.621
1,2015-01-05,5.424
2,2015-01-06,5.329
3,2015-01-07,5.224
4,2015-01-08,5.453


In [10]:
## Filtrer la période entre janvier 2015 et décembre 2016
df_period = df[(df['Date'] >= "2015-01-01") & (df['Date'] <= "2016-12-31")]
df_period.head()

,Date,Cours de Natixis
0,2015-01-02,5.621
1,2015-01-05,5.424
2,2015-01-06,5.329
3,2015-01-07,5.224
4,2015-01-08,5.453


In [12]:
df_period['Rendement'] = (
    df_period['Cours de Natixis'].shift(-1) - df_period['Cours de Natixis']
) / df_period['Cours de Natixis']


C:\Users\emmab\AppData\Local\Temp\ipykernel_31044\2369067441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_period['Rendement'] = (


In [14]:
df_period.head()

,Date,Cours de Natixis,Rendement
0,2015-01-02,5.621,-0.035047
1,2015-01-05,5.424,-0.017515
2,2015-01-06,5.329,-0.019704
3,2015-01-07,5.224,0.043836
4,2015-01-08,5.453,-0.020723


## Classer les rendements par ordre croissant

In [17]:
df_sorted = df_period.sort_values(by='Rendement', ascending=True)

In [19]:
df_sorted.head()

,Date,Cours de Natixis,Rendement
377,2016-06-23,4.150,-0.171325
344,2016-05-09,4.386,-0.069083
357,2016-05-26,4.752,-0.061448
404,2016-08-01,3.643,-0.060664
276,2016-02-01,4.470,-0.058613


In [21]:
df_sorted.tail()

,Date,Cours de Natixis,Rendement
304,2016-03-10,4.780,0.063389
163,2015-08-24,5.405,0.064755
402,2016-07-28,3.425,0.075912
325,2016-04-12,4.437,0.078431
512,2016-12-30,5.360,NaN


In [23]:
# Choisir alpha
alpha = 0.05
n = len(df_sorted)

# Trouver l'indice correspondant
k = int(np.floor(alpha * (n - 1)))
quantile_row = df_sorted.iloc[k]

print(f"Quantile {alpha:.2f} basé sur le tri :")
print(quantile_row[['Date', 'Cours de Natixis', 'Rendement']])

Quantile 0.05 basé sur le tri :
Date                2016-05-02 00:00:00
Cours de Natixis                  4.726
Rendement                     -0.038934
Name: 339, dtype: object


## VaR based on a parametric distribution (say Gaussian)

In [26]:
mu = df_sorted["Rendement"].mean()
sigma=df_sorted["Rendement"].std()

In [28]:
from scipy.stats import norm
confidence_level = 0.95
z = norm.ppf(1 - confidence_level)  
VaR = -(mu + z * sigma)

# --- 5. Affichage ---
print(f"VaR (paramétrique, {int(confidence_level*100)}%) = {VaR:.4%}")

VaR (paramétrique, 95%) = 3.9097%


## VaR based on a non-parametric distribution (Gaussian Kernel)

In [31]:
from scipy.stats import gaussian_kde

returns = df_sorted["Rendement"].dropna().values
kde = gaussian_kde(returns, bw_method='scott') 
simulated = kde.resample(100000)[0]
confidence_level = 0.95
alpha = 1 - confidence_level
VaR_nonparam = -np.percentile(simulated, alpha * 100)

# --- 6. Résultats ---
print(f"VaR (non-paramétrique KDE, {int(confidence_level*100)}%) = {VaR_nonparam:.4%}")

VaR (non-paramétrique KDE, 95%) = 3.9492%


In [33]:
df_sorted["Exceed_Param"] = df_sorted["Rendement"] < -VaR
df_sorted["Exceed_NonParam"] = df_sorted["Rendement"] < -VaR_nonparam

# --- 6. Compute proportions of breaches ---
prop_param = df_sorted["Exceed_Param"].mean()
prop_nonparam = df_sorted["Exceed_NonParam"].mean()

# --- 7. Display results ---
print(f"In-sample validation (Jan 2015 – Dec 2016)")
print(f"Parametric VaR (95%): {VaR:.4%} | Breaches: {prop_param:.2%}")
print(f"Non-parametric VaR (95%): {VaR_nonparam:.4%} | Breaches: {prop_nonparam:.2%}")

In-sample validation (Jan 2015 – Dec 2016)
Parametric VaR (95%): 3.9097% | Breaches: 4.48%
Non-parametric VaR (95%): 3.9492% | Breaches: 4.29%


In [45]:
df['Rendement'] = (
    df['Cours de Natixis'].shift(-1) - df['Cours de Natixis']
) / df['Cours de Natixis']
## Filtrer la période entre janvier 2015 et décembre 2016
df_period2 = df[(df['Date'] >= "2017-01-01") & (df['Date'] <= "2018-12-31")]
df_period2.head()

,Date,Cours de Natixis,Rendement,Exceed_Param,Exceed_NonParam
513,2017-01-02,5.400,0.040741,False,False
514,2017-01-03,5.620,0.003737,False,False
515,2017-01-04,5.641,-0.008155,False,False
516,2017-01-05,5.595,-0.005719,False,False
517,2017-01-06,5.563,-0.020672,False,False


In [41]:
df_period2["Exceed_Param"] = df_period2["Rendement"] < -VaR
df_period2["Exceed_NonParam"] = df_period2["Rendement"] < -VaR_nonparam

# --- 6. Compute proportion of breaches ---
prop_param = df_period2["Exceed_Param"].mean()
prop_nonparam = df_period2["Exceed_NonParam"].mean()

# --- 7. Print results ---
print(f"Out-of-sample validation (Jan 2017 – Dec 2018)")
print(f"Parametric VaR (95%): {VaR:.4%} | Breaches: {prop_param:.2%}")
print(f"Non-parametric VaR (95%): {VaR_nonparam:.4%} | Breaches: {prop_nonparam:.2%}")

Out-of-sample validation (Jan 2017 – Dec 2018)
Parametric VaR (95%): 3.9097% | Breaches: 1.57%
Non-parametric VaR (95%): 3.9492% | Breaches: 1.57%


C:\Users\emmab\AppData\Local\Temp\ipykernel_31044\1726021188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_period2["Exceed_Param"] = df_period2["Rendement"] < -VaR
C:\Users\emmab\AppData\Local\Temp\ipykernel_31044\1726021188.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_period2["Exceed_NonParam"] = df_period2["Rendement"] < -VaR_nonparam
